# Exploratory Data Analysis
I'm a bit more comfortable exploring datasets in Python, so this is my work area for making sure my I/O and data transformations are working properly.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import requests
import json
import yaml

### Loading data from API

In [17]:
# Test connection
with open('leaflet_dash/auth.yaml') as auth:
    token = yaml.safe_load(auth)['app_token']

url = f'https://data.cityofchicago.org/resource/kc9i-wq85.geojson?$$app_token={token}&$limit=999999'
r = requests.get(url)
print(r.status_code)

200


In [18]:
jd = json.JSONDecoder()
load = jd.decode(r.text)
load.keys()

dict_keys(['type', 'features', 'crs'])

In [19]:
len(load['features'])

4978

In [22]:
type(load['features'])

list

In [25]:
load['features'][0]

{'type': 'Feature',
 'geometry': {'type': 'Point',
  'coordinates': [-87.62766855145455, 41.72433163710899]},
 'properties': {'issued_date': '2011-03-07T00:00:00.000',
  'disposition_description': 'City Non suit - Motion to set-aside default - Granted',
  'latitude': '41.72433163710899',
  'total_administrative_costs': '0',
  'entity_or_person_s_': 'JP MORGAN CHASE, ',
  'total_paid': '0',
  'last_hearing_date': '2011-08-05T11:00:00.000',
  'total_fines': '0',
  'current_amount_due': '0',
  'longitude': '-87.62766855145455',
  'docket_number': '11CP015666',
  'original_total_amount_due': '0',
  'collection_costs_or_attorney_fees': '0',
  'property_address': '9332 S LASALLE ',
  'court_cost': '0',
  'issuing_department': 'POLICE',
  'violation_type': '13-12-125  Duty to secure and maintain vacant building.|13-12-140  Watchman required',
  'violation_number': 'P002161520|P002161521',
  'interest_amount': '0'}}

In [33]:
df_url = f'https://data.cityofchicago.org/resource/kc9i-wq85.json?$$app_token={token}&$limit=999999'
pd.read_json(df_url).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4978 entries, 0 to 4977
Data columns (total 25 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   docket_number                      4978 non-null   object 
 1   violation_number                   4978 non-null   object 
 2   issued_date                        4978 non-null   object 
 3   issuing_department                 4978 non-null   object 
 4   last_hearing_date                  4978 non-null   object 
 5   property_address                   4978 non-null   object 
 6   violation_type                     4978 non-null   object 
 7   entity_or_person_s_                4978 non-null   object 
 8   disposition_description            4978 non-null   object 
 9   total_fines                        4978 non-null   int64  
 10  total_administrative_costs         4978 non-null   int64  
 11  interest_amount                    4978 non-null   float

In [36]:
chi_vacant_abandoned = pd.read_json(df_url).iloc[:,:19]
chi_vacant_abandoned.head()

,docket_number,violation_number,issued_date,issuing_department,last_hearing_date,property_address,violation_type,entity_or_person_s_,disposition_description,total_fines,total_administrative_costs,interest_amount,collection_costs_or_attorney_fees,court_cost,original_total_amount_due,total_paid,current_amount_due,latitude,longitude
0,11CP015666,P002161520|P002161521,2011-03-07T00:00:00.000,POLICE,2011-08-05T11:00:00.000,9332 S LASALLE,13-12-125 Duty to secure and maintain vacant ...,"JP MORGAN CHASE,",City Non suit - Motion to set-aside default - ...,0,0,0.00,0.00,0.00,0.00,0.0,0.0,41.724332,-87.627669
1,11CP020396,P002513827,2011-05-04T00:00:00.000,POLICE,2011-07-01T14:30:00.000,2107 N SAWYER,13-12-140 Watchman required,"DEUTSCHE BANK,",Liable - By plea,100,0,0.00,0.00,0.00,140.00,280.0,-140.0,41.919469,-87.708844
2,11CP024049,P002165386|P002165387,2011-05-03T00:00:00.000,POLICE,2011-07-01T14:00:00.000,12863 S NORMAL,13-12-125 Duty to secure and maintain vacant ...,"WELLS FARGO BANK NA,",City non-suit,0,0,0.00,0.00,0.00,0.00,0.0,0.0,41.659662,-87.633937
3,11CP026554,P002109256,2011-05-27T00:00:00.000,POLICE,2011-07-01T09:00:00.000,5700 S ABERDEEN,13-12-140 Watchman required,"OCWEN LOAN SERVICING LLC,",Not liable - City failed to meet burden of proof,800,0,27.96,236.88,26.09,1130.93,1125.0,0.0,41.790474,-87.652497
4,11CP054441,P003018262|P003018263|P003018264,2011-08-25T00:00:00.000,POLICE,2011-10-07T09:00:00.000,5035 S RACINE,13-12-140 Watchman required|13-12-140 Watchm...,"WELLS FARGO,",City non-suit,0,0,0.00,0.00,0.00,0.00,0.0,0.0,41.802223,-87.654932


## Exploring data
### Violation types

In [37]:
# How many types of violations are there?
chi_vacant_abandoned.violation_type.nunique()

44

I think this is too many distinct types to be a useful dashboard filter. But I'll include it in the tooltip, I guess.

### Dispositions

In [38]:
# How many unique disposition descriptions are there?
chi_vacant_abandoned.disposition_description.nunique()

15

In [39]:
# What are the unique disposition descriptions?
chi_vacant_abandoned.disposition_description.unique()

array(['City Non suit - Motion to set-aside default - Granted',
       'Liable - By plea', 'City non-suit',
       'Not liable - City failed to meet burden of proof',
       'Not liable - City failed to establish prima facie case',
       'Default - Liable by prove-up',
       'Dismissed for want of prosecution - No service',
       'Vendor scanning error',
       'Liable - By Plea - Motion to set-aside default granted',
       'Not liable - Respondent came into compliance with building code prior to hearing',
       'Not liable - City failed to establish Prima Facie case - Motion to set-aside default granted',
       'Liable - By contested finding',
       'Dismissed for want of prosecution - No service - Motion to set-aside default - Granted',
       'Denied - Motion to set aside denied - Prior default order still stands',
       'Denied - Motion to set-aside default - lack of good cause'],
      dtype=object)

In [51]:
# Splitting disposition descriptions into their header
chi_vacant_abandoned.disposition_description[0].split(' - ')[0]

'City Non suit'

In [52]:
# Create disposition header column by applying above function
chi_vacant_abandoned.loc[:,'disposition_header'] = \
    chi_vacant_abandoned.loc[:,'disposition_description'].apply(lambda x: x.split(' - ')[0])

In [54]:
# How many disposition headers is that?
chi_vacant_abandoned.disposition_header.nunique()

8

In [55]:
# How common is each disposition header?
chi_vacant_abandoned.disposition_header.value_counts()

City non-suit                        3997
Default                               596
City Non suit                         146
Liable                                138
Not liable                             68
Dismissed for want of prosecution      27
Vendor scanning error                   4
Denied                                  2
Name: disposition_header, dtype: int64

You know what, I don't think any of this is too important for filtering. I'll leave the full disposition for the tooltip.

### Issuing departments

In [59]:
# What are the issuing departments??
chi_vacant_abandoned.issuing_department.value_counts()

POLICE     4961
BLDINGS      17
Name: issuing_department, dtype: int64

Tooltip.

### Entity or person

In [62]:
chi_vacant_abandoned.entity_or_person_s_.nunique()

270

I think that's _way_ too many to be useful as a filter.

### Amount due

In [60]:
# What do the original and current amount due fields look like?
chi_vacant_abandoned.current_amount_due.describe()

count     4978.000000
mean       145.077045
std        941.637275
min      -4636.800000
25%          0.000000
50%          0.000000
75%          0.000000
max      13220.960000
Name: current_amount_due, dtype: float64

In [61]:
chi_vacant_abandoned.original_total_amount_due.describe()

count     4978.000000
mean       560.592676
std       1492.890630
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max      13353.440000
Name: original_total_amount_due, dtype: float64

## Practical data questions

In [63]:
# What's the minimum issued date?
chi_vacant_abandoned.issued_date.min()

'2001-01-01T00:00:00.000'

In [64]:
# Who are the entities with the highest outstanding fine balance?
chi_vacant_abandoned[[
    'entity_or_person_s_','current_amount_due'
]].groupby('entity_or_person_s_', as_index=False).sum().sort_values('current_amount_due', ascending=False).head(10)

,entity_or_person_s_,current_amount_due
45,"CHASE REO GROUP LLC,",145700.00
205,"REO PARTNERS LLC,",121771.40
183,"NATIONWIDE REO LLC,",89493.75
102,"FEDERAL NATL MTG ASSN,",70255.19
203,"REO DIRECT LLC,",52373.09
44,"CHASE REO GROUP LLC C/O ANTWAN REID,",40000.00
177,"NATIONAL ASSET MANAGEMENT GROUP LLC,",35303.24
153,"JNL FINANCIAL LLC,",28937.96
154,"JNL FINANCIAL,",20221.18
192,"ONE WEST BANK FSB,",19238.51


In [71]:
chi_vacant_abandoned[
    chi_vacant_abandoned.entity_or_person_s_ == 'BANK OF AMERICA, '
]

,docket_number,violation_number,issued_date,issuing_department,last_hearing_date,property_address,violation_type,entity_or_person_s_,disposition_description,total_fines,total_administrative_costs,interest_amount,collection_costs_or_attorney_fees,court_cost,original_total_amount_due,total_paid,current_amount_due,latitude,longitude,disposition_header
19,12CP058877,P003180744|P003180745,2012-07-30T00:00:00.000,POLICE,2012-09-07T13:00:00.000,3719 W OHIO,13-12-125 Duty to secure and maintain vacant ...,"BANK OF AMERICA,",City non-suit,0,0,0.00,0.0,0.0,0.00,0.00,0.0,41.891642,-87.719529,City non-suit
25,13CP009146,P003766968|P003766969,2013-02-01T00:00:00.000,POLICE,2013-03-01T13:00:00.000,4216 W ADAMS,13-12-125 Duty to secure and maintain vacant ...,"BANK OF AMERICA,",City non-suit,0,0,0.00,0.0,0.0,0.00,0.00,0.0,41.878019,-87.731215,City non-suit
50,14CP125031,P003839420,2014-09-13T00:00:00.000,POLICE,2014-11-05T09:00:00.000,201 S STATE,7-24-099(a) Cannabis - up to 15 grams,"BANK OF AMERICA,",Vendor scanning error,0,0,0.00,0.0,0.0,0.00,0.00,0.0,41.879431,-87.627563,Vendor scanning error
58,15CP080441,P004806688|P004806689,2015-05-08T00:00:00.000,POLICE,2015-08-07T11:00:00.000,31 W 110TH,13-12-125 Duty to secure and maintain vacant ...,"BANK OF AMERICA,",City non-suit,0,0,0.00,0.0,0.0,0.00,0.00,0.0,NaN,NaN,City non-suit
60,15CP114089,P003124666|P003124667,2015-08-11T00:00:00.000,POLICE,2015-10-02T09:00:00.000,540 W GARFIELD,13-12-125 Duty to secure and maintain vacant ...,"BANK OF AMERICA,",City non-suit,1400,0,2.13,0.0,0.0,1442.13,1440.00,0.0,41.794517,-87.639286,City non-suit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4897,15CP006384,P004593378|P004593379|P004593380|P004593381|P0...,2014-11-15T00:00:00.000,POLICE,2015-02-06T09:00:00.000,4155 W 78,13-12-125 Duty to secure and maintain vacant ...,"BANK OF AMERICA,",Default - Liable by prove-up,4300,0,151.37,0.0,0.0,4491.37,4490.75,0.0,41.751811,-87.724843,Default
4937,12CP063387,P003349139|P003349140|P003349141|P003349142|P0...,2012-08-29T00:00:00.000,POLICE,2012-10-05T09:00:00.000,4004 S MAPLEWOOD,13-12-125 Duty to secure and maintain vacant ...,"BANK OF AMERICA,",City non-suit,0,0,0.00,0.0,0.0,0.00,0.00,0.0,41.820881,-87.688613,City non-suit
4944,15CP158679,P004916840|P004916841|P004916842|P004916843|P0...,2015-12-09T00:00:00.000,POLICE,2016-02-05T09:00:00.000,5752 S SANGAMON,13-12-125 Duty to secure and maintain vacant ...,"BANK OF AMERICA,",City non-suit,0,0,0.00,0.0,0.0,0.00,0.00,0.0,41.789052,-87.648815,City non-suit
4958,12CP002489,P003141012|P003141013|P003141014|P003141015|P0...,2011-12-26T00:00:00.000,POLICE,2012-02-03T09:00:00.000,5022 S CARPENTER,13-12-125 Duty to secure and maintain vacant ...,"BANK OF AMERICA,",City non-suit,0,0,0.00,0.0,0.0,0.00,0.00,0.0,41.802606,-87.651590,City non-suit


In [ ]:
chi_vacant_abandoned